In [1]:
from pathlib import PosixPath
import dask
import dask.dataframe as dd
import numpy as np

classes = ['retweet', 'reply', 'like', 'retweet_with_comment']
label_cols = ['has_' + c for c in classes]
cols = ['e{:03d}'.format(i) for i in range(768)] + ['user_id']

def compute_user_embeddings(data_path, ds_name):
    data_dir = PosixPath(data_path).expanduser()
    input_file = data_dir / f"{ds_name}.parquet/"
    output_file = data_dir / f"{ds_name}_userembeddings.parquet/"
    df = dd.read_parquet(str(input_file))
    print(df.columns)
    df = df[df[label_cols].any(axis=1)][cols].groupby('user_id').mean().astype(np.float32)
    df.repartition(partition_size='100MB').to_parquet(output_file)
    df = None
    

In [2]:
compute_user_embeddings('~/recsys2020', 'train')

Index(['user_id', 'tweet_id', 'tweet_timestamp', 'author_follower_count',
       'author_following_count', 'user_follower_count', 'user_following_count',
       'num_hashtags', 'num_media', 'num_links',
       ...
       'e762', 'e763', 'e764', 'e765', 'e766', 'e767', 'has_retweet',
       'has_reply', 'has_like', 'has_retweet_with_comment'],
      dtype='object', length=786)


In [5]:
dd.read_parquet('~/recsys2020/train_userembeddings.parquet').count().compute()

e000    99836
e001    99836
e002    99836
e003    99836
e004    99836
        ...  
e763    99836
e764    99836
e765    99836
e766    99836
e767    99836
Length: 768, dtype: int64